# Trabalho de paralelismo
### Simulação de Monte Carlo para aproximação de PI
--- 

## Introdução ao problema

A simulação de Monte Carlo é uma técnica estatística que utiliza amostragem aleatória para prever resultados em situações incertas permitindo análise mais fundamentada de riscos e proporcionando a tomada de decisões mais informadas.

A técnica de Monte Carlo será utilizada para o seguinte problema de natureza geométrica: dado um quadrado de lado igual a uma unidade (1) desenha-se dentro dele um quarto de um círculo.

![Imagem Ilustrativa do problema](/assets/image.png)

Ao inserir pontos aletórios dentro do quadrado, alguns pontos podem estar dentro do círculo enquanto outros podem estar fora. A proporção de pontos dentro do círculo em relação ao total é igual a razão entre a área do círculo e a área do quadrado. 

Em termos matemáticos:

$$
\text{ Área do círculo (contida no quadrado) } = \frac{πr^{2}}{4}, r = 1
$$
$$
\text{ Área do quadrado } = 1^2 = 1
$$
$$
\frac{\text{ Área do círculo }}{\text{ Área do quadrado }} = \frac{π}{4}
$$

Logo, se **P** é a probabilidade de um ponto cair dentro do círculo então:

$$
P ≈ \frac{π}{4}
$$
$$
π ≈ 4P  
$$
$$
\text{ ou }
$$
$$
 π ≈ 4 \frac{\text{ Nº de pontos dentro do círculo }}{\text{ Total de pontos }}
$$

Portanto, cabe usar a técnica de Monte Carlo para geração de pontos aleatórios com fim de estimar o valor de π. A seguir serão apresentadas as abordagens serial e paralelizada do problema seguidas de análises a respeito da eficiência de cada implementação.

---

## Implementação

```python
# ================================
# FUNÇÃO
# ================================
import random

def contar_pontos_circulo(num_pontos):
    dentro_circulo = 0
    for _ in range(num_pontos):
        x = random.random()
        y = random.random()
        if x*x + y*y <= 1:
            dentro_circulo += 1
    return dentro_circulo


### Breve explicação: equação do círculo

A condicional presente no método de contagem dos pontos é baseada na seguinte équação de círculo de raio **r** centrado na origem (0,0):
$$
x^2 + y^2 = r^2
$$

Isso significa que qualquer ponto no plano que satisfaz a seguinte inequação está dentro ou sobre o círculo para r = 1:
$$
x^2 + y^2 \le 1
$$


## Versão serial

```python
import time

if __name__ == "__main__":
    total_pontos_serial = 250_000_000

    inicio_serial = time.time()
    dentro_circulo_total_serial = contar_pontos_circulo(total_pontos_serial)
    pi_serial = 4 * dentro_circulo_total_serial / total_pontos_serial
    fim_serial = time.time()

    print(f"π ≈ {pi_serial} (serial)")
    print(f"Tempo serial: {fim_serial - inicio_serial:.2f} segundos")


### Avaliação da Execução Serial

**1. Qual foi o tempo de execução do seu algoritmo em modo serial? Como você
mediu esse tempo?** 
> O tempo de execução resultante obtido foi cerca de 86,04 segundos para 250 milhões de pontos gerados. O tempo de exeucução foi medido usando a biblioteca `time` do python da seguinte maneira: o tempo de íncio foi registrado na variável de nome `incio_serial` seguido da chamada do método `contar_pontos_circulo()` para calcular o valor estimado de pi logo após. Por fim, foi registrado o tempo atual na variável `fim_serial` e o tempo serial foi apresentado no final do código como uma diferença entre o tempo inicial e o tempo final.

**2. Quais aspectos do algoritmo você percebe que se beneficiariam de uma
abordagem paralela?** 
> Para uma quantidade de pontos suficientemente grande, a tarefa de contagem dos pontos dentro do círculo pode se beneficiar com a implementação do paralelismo visto que possui complexidade linear que cresce conforme o número de pontos gerados aumenta e esse método define a complexidade do código, pois as demais linhas de código são consideravelmente simples. Além disso, a geração dos pontos dentro do método é independente, podendo ser facilmente paralelizada.

**3. Você utilizou alguma técnica de otimização no código serial para reduzir o
tempo de execução? Se sim, quais?**
> Não houve otimização do código serial

**4. O seu código serial apresenta alguma limitação de desempenho que poderia ser
resolvida por computação paralela? Explique.**
> Sim, a quantidade de pontos gerados e verificados pode ser dividida entre processos para diminuir o tempo de execução do código.

**5. Como você validou os resultados da execução serial? Existe alguma verificação
de consistência no seu código?**
> Atualmente, a validação é implícita, baseada na proximidade do resultado com π esperado.

**6. Você teve dificuldades ao implementar o código serial? Quais desafios você
enfrentou?**
> Não houveram dificuldades durante a implementação do código serial. No entanto, houve um desafio inicial durante o planejamento do código com base na problemática abordada.

**7. Quais foram os principais bottlenecks identificados na execução serial do seu
programa?**
> Os principais gargalos que limitam o desempenho do código estão na geração dos pontos aleatórios e verificação de pontos dentro do círculo dentro laço feito para contabilizar os pontos que possuem complexidade linear relacionada à quantidade de pontos que se deseja gerar

**8. O que você aprendeu sobre a relação entre a eficiência da execução serial e o
uso de recursos computacionais (tempo de CPU, memória)?**
> Foi observado que dentro da execução serial, a carga total do código é sequencialmente computacionada e o tempo total cresce linearmente conforme o número de pontos aumenta. Já a memória não aparentou ser um fator prejudicial para a eficiência do código.

---

## Versão paralela

```python
from multiprocessing import Pool
import time

if __name__ == "__main__":
    total_pontos = 250_000_000
    n_processos = 4
    pontos_por_processo = total_pontos // n_processos

    inicio_paralelo = time.time()
    with Pool(n_processos) as pool:
        resultados = pool.map(contar_pontos_circulo, [pontos_por_processo]*n_processos)

    dentro_circulo_total = sum(resultados)
    pi_paralelo = 4 * dentro_circulo_total / total_pontos
    fim_paralelo = time.time()

    print(f"π ≈ {pi_paralelo} (paralelo)")
    print(f"Tempo paralelo: {fim_paralelo - inicio_paralelo:.2f} segundos")


### Avaliação da execução paralela

**9. Qual foi o tempo de execução do seu algoritmo em modo paralelo? Como você
mediu esse tempo?**
> O tempo de execução do algoritmo em modo paralelo foi consideravelmente menor do que no modo serial, dependendo do número de processos utilizados. No caso com 4 processos, o tempo de execução obtido foi cerca de 23,31 segundos para 250 milhões de pontos gerados. O tempo foi medido utilizando a biblioteca `time` do Python, da seguinte forma: o tempo de início foi registrado na variável `inicio_paralelo` imediatamente antes de iniciar a execução dos processos com o `Pool.map()`. Após a conclusão do cálculo e soma dos resultados parciais, o tempo atual foi registrado na variável `fim_paralelo`. O tempo total de execução paralela foi então obtido como a diferença entre `fim_paralelo` e `inicio_paralelo` e apresentado no final do código

**10. Quais recursos você utilizou para implementar o paralelismo? (Ex: threads,
processos, multiprocessing, etc.)**
> A biblioteca multiprocessing do python foi utilizada para implementação do paralelismo. Utilizamos a classe Pool que permite criar uma pool de processos para executar as tarefas em paralelo. Além disso utilizou-se processos ao invés de threads na implementação do código.

**11. Como você adaptou o código serial para uma execução paralela? Houve alguma
reestruturação significativa?**
> Sim, para a implementação de um modelo paralelizado, houve a necessidade de se definir variáveis do número de processos e de pontos por processo. Em seguida, `with Pool(n_processos) as pool:` cria uma instância que recebe o número de processos como argumento da variável `pool` e então `pool.map()` recebe como primeiro parâmetro a função a ser executada e como segundo parâmetro a lista de de valores que serão passados para cada processo (a carga está balanceada). Por fim a variável `dentro_circulo_total` agrega os valores obtidos por cada processo.

**12. Você implementou alguma técnica de sincronização entre os processos ou
threads paralelos? Como ela foi aplicada?
**
> Não há sincronização implementada visto que na problemática que a paralelização se propõe a resolver, os valores podem ser obtidos independentemente pois não há risco de conflitos.

**13. Quais benefícios você observou ao aplicar a computação paralela em relação à
execução serial?
**
> O tempo de execução foi consideravelmente menor em relação à implementação serial. 

**14. Como a metodologia de Foster influenciou sua abordagem para a
implementação paralela? Você seguiu os princípios dessa metodologia?
**
> A metodologia influenciou na implementação principalmente com relação à divisão das tarefas em subproblemas independentes, Além do paralelismo de dados ao dividir a carga entre 4 processadores de forma balanceada.

**15. O que você aprendeu com a comparação de desempenho entre a execução serial
e paralela? Como a diferença de tempo de execução se reflete no desempenho do
sistema paralelo?**
> Os resultados obtidos a partir das implementações serial e paralela mostram que para cenários como o de simulação de Monte Carlo, onde se costuma ter valores suficientemente grandes para gerar boas estimações estatísticas, a abordagem paralela aumenta o desempenho pela diminuição do tempo de execução.